In [1]:
import os

# Das aktuelle Arbeitsverzeichnis muss das sein, indem sich die Module befinden
# Das ist dormalerweise der Fall, wenn das notebook direkt aus dem Ordner geöffnet wird
# print(f"Das aktuelle Arbeitsverzeichnis {os.getcwd()}")
# sonst: gib den Ordner indem indem die module gespeichert sind an, relativ zum aktuellen Arbeitsverzeichnis:
# os.chdir("pfad/zum/neuen/Verzeichnis")

from icecream import ic
import numpy as np

import graphviz
from print_graph import print_graph

from forward_backward_functions_and_nodes import *


In [2]:
sin = Sin()
ic(sin, sin.forward_func, sin.backward_func)
f1, f2 = sin.get_functions()
ic(f1, f2)

ic(Multiply().forward_func)
ic(Multiply(allow_arbitrary_many=True).forward_func)
ic(Multiply(allow_arbitrary_many=False).forward_func)


ic| sin: <forward_backward_functions_and_nodes.Sin object at 0x72fa70f2c6e0>
    sin.forward_func: <ufunc 'sin'>
    sin.backward_func: <ufunc 'cos'>
ic| f1: <ufunc 'sin'>, f2: <ufunc 'cos'>
ic| Multiply().forward_func: <ufunc 'multiply'>
ic| Multiply(allow_arbitrary_many=True).forward_func: <bound method Multiply.<lambda> of <forward_backward_functions_and_nodes.Multiply object at 0x72fa701c2ab0>>
ic| Multiply(allow_arbitrary_many=False).forward_func: <ufunc 'multiply'>


<ufunc 'multiply'>

In [3]:
x = Expr_end_node(np.float64(0.345))
sin = Sin()
example_expr1 = sin( sin( sin( x ) )) 

#sin(sin(sin(x)))
"""
sin
 |
sin
 |
sin
 |
 x=0.345
"""


#cos(a)*sin(b + tanh(c))
example_expr2 = Expr_node(Multiply(), 
                          [Expr_node(Cos(),
                                     [Expr_end_node(np.float64(0.345))]
                                    ), 
                           Expr_node(Sin(),
                                     [Expr_node(Add(), 
                                                [Expr_end_node(np.float64(0.345)),
                                                 Expr_node(Tanh(), 
                                                           [Expr_end_node(np.float64(0.345))]
                                                          )
                                                ]
                                               )
                                     ]
                                    )
                          ]
                         )

# the same using the __call__ method of function (see above in function definition):
example_expr3 = Multiply()(
                        Cos()(
                            Expr_end_node(np.float64(0.345))
                        ),
                        Sin()(
                            Add()(
                                Expr_end_node(np.float64(0.345)),
                                Tanh()(
                                    Expr_end_node(np.float64(0.345))
                                )
                            )
                        )
                )
"""
    multiply
    |      |
   cos    sin
   |       |
  a=0.345 add
          | |
    b=0.345 tanh
             |
         c=0.345
"""



'\n    multiply\n    |      |\n   cos    sin\n   |       |\n  a=0.345 add\n          | |\n    b=0.345 tanh\n             |\n         c=0.345\n'

In [46]:

graph1 = graphviz.Digraph('graph1', comment='test') 
graph1.attr(rankdir="LR")
print_graph(example_expr1, graph1)
#doctest_mark_exe() 
graph1.render(directory='graph_out/tt', view=True)



'graph_out/tt/graph1.gv.pdf'

In [44]:
graph2 = graphviz.Digraph('graph2', comment='test') 
graph2.attr(rankdir="TD")
print_graph(example_expr2, graph2)
#doctest_mark_exe() 
graph2.render(directory='graph_out/tt', view=True)

'graph_out/tt/graph2.gv.pdf'

In [45]:
graph3 = graphviz.Digraph('graph3', comment='test') 
graph3.attr(rankdir="LR")
print_graph(example_expr3, graph3)
#doctest_mark_exe() 
graph3.render(directory='graph_out/tt', view=True)

'graph_out/tt/graph3.gv.pdf'

In [38]:
#f(g(h(x)))' = f'(g(h(x))) * (g(h(x)))' =  f'(g(h(x))) * g'(h(x)) * h'(x)
# h'(x) * g'(h(x)) * f'(g(h(x)))

def forward(node):
    return node.forward_func(*(forward(child) for child in node.childs)) if type(node) is not Expr_end_node else node.value 
"""
def forward(node):
    if type(node) is not Expr_end_node:
    
        node.forward_func(*(forward(child) for child in node.childs))
    else:
        node.value 
"""

def backward(node, value = np.float64(1)):
    if type(node) is not Expr_end_node:
        child_values = [forward(child) for child in node.childs]
        if len(node.childs) == 1:
            new_value = node.backward_func(*child_values)
            if value.ndim == 0 or new_value.ndim == 0:
                backward(node.childs[0], value * new_value)
            else:
                backward(node.childs[0], value @ new_value) # @ is matrix product
            
        else:
            for child, new_value in zip(node.childs, node.backward_func(*child_values), strict=True):
                if value.ndim == 0 or new_value.ndim == 0:
                    backward(child, value * new_value)
                else:
                    backward(child, value @ new_value) # @ is matrix product
                    
    else: 
        node.grad_value += value
        

In [41]:
ex1 = example_expr1
forward(ex1)

x.grad_value=0

ic(forward(ex1))
ic(np.sin(np.sin(np.sin(np.float64(.345)))))

backward(ex1)
ic(x.grad_value)
ic(np.cos(np.sin(np.sin(.345)))*np.cos(np.sin(.345))*np.cos(0.345))
print(x.grad_value)

ic| forward(ex1): np.float64(0.32573257710464837)
ic| np.sin(np.sin(np.sin(np.float64(.345)))): np.float64(0.32573257710464837)
ic| x.grad_value: np.float64(0.8393506506551831)
ic| np.cos(np.sin(np.sin(.345)))*np.cos(np.sin(.345))*np.cos(0.345): np.float64(0.8393506506551831)


0.8393506506551831


In [42]:
m_product = Matrix_vector_product()
mat1 = Expr_end_node(np.random.rand(3,2))
mat2 = Expr_end_node(np.random.rand(1,3))
vec = Expr_end_node(np.random.rand(2))
a = m_product(mat2, m_product(mat1, vec))
ic(forward(a))
backward(a)
ic(mat1.value, mat1.grad_value)
ic(mat2.value, mat2.grad_value)
ic(vec.value, vec.grad_value)

ic| forward(a): array([0.51682058])
ic| mat1.value: array([[0.9004706 , 0.11569409],
                       [0.87386324, 0.08213585],
                       [0.78566756, 0.19352601]])
    mat1.grad_value: array([[[0.2046392 , 0.42078148]],
                     
                            [[0.03443949, 0.07081488]],
                     
                            [[0.20949277, 0.43076146]]])
ic| mat2.value: array([[0.72342617, 0.12174808, 0.74058418]])
    mat2.grad_value: array([[[0.32201422, 0.29496848, 0.33481031]]])
ic| vec.value: array([0.28287503, 0.5816509 ])
    vec.grad_value: array([[1.33966813, 0.23701832]])


(array([0.28287503, 0.5816509 ]), array([[1.33966813, 0.23701832]]))